# Robustness Analysis: Adjoint and Toggle-Frame Objectives Subject to Either Multiplicative or Additive Errors

This notebook compares the performance of the adjoint and toggling-frame robustness objectives for multiplicative and additive error terms in the system's Hamiltonian. 

## Imports

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using NamedTrajectories
using Statistics
using CairoMakie
using Random
using ColorSchemes
using Makie
using Printf

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
┌ Warning: Circular dependency detected.
│ Precompilation will be skipped for dependencies in this cycle:
│  ┌ Piccolissimo
│  └─ QuantumCollocation
└ @ Base.Precompilation precompilation.jl:651
┌ Warning: Circular dependency detected.
│ Precompilation will be skipped for dependencies in this cycle:
│  ┌ Piccolissimo
│  └─ QuantumCollocation
└ @ Base.Precompilation precompilation.jl:651


In [2]:
# Problem parameters
T = 20
Δt = 0.2
U_goal = GATES.H
H_drive = [PAULIS.X, PAULIS.Y, PAULIS.Z]
piccolo_opts = PiccoloOptions(verbose=false)
pretty_print(X::AbstractMatrix) = Base.show(stdout, "text/plain", X);
sys = QuantumSystem(H_drive)
# Adjoint
∂ₑHₐ = [PAULIS.X, PAULIS.Y, PAULIS.Z]
varsys = VariationalQuantumSystem(
    H_drive,
    ∂ₑHₐ
)


VariationalQuantumSystem: levels = 2, n_drives = 3

Setup metrics

In [3]:
function SpaceCurve(traj::NamedTrajectory, U_goal::AbstractMatrix{<:Number}, H_err::AbstractMatrix{<:Number})
    T = traj.T
    first_order_terms = Vector{Matrix{ComplexF64}}(undef, T)
    first_order_integral = zeros(ComplexF64, size(U_goal))

    for i in 1:T
        U = iso_vec_to_operator(traj.Ũ⃗[:, i])
        first_order_integral += U' * H_err * U
        first_order_terms[i] = first_order_integral
    end
    d = size(U_goal)[1]
    space_curve = [[real(tr(PAULIS.X * first_order_terms[t] / (d * T))),
                    real(tr(PAULIS.Y * first_order_terms[t] / (d * T))),
                    real(tr(PAULIS.Z * first_order_terms[t] / (d * T)))] for t in 1:T] 
    return space_curve
end

function space_curve_robustness(traj::NamedTrajectory, U_goal::AbstractMatrix, H_err::AbstractMatrix)
    curve = SpaceCurve(traj, U_goal, H_err)
    # Use the norm of the final point as a robustness measure
    # (larger values indicate more accumulated error sensitivity)
    final_point = curve[end]
    return norm(final_point)
end

function width_robustness(system::AbstractQuantumSystem, traj::NamedTrajectory; thresh::Float64=0.999)
    F = 1.0
    drift = system.H.H_drift
    drive = system.H.H_drives
    pauls = [PAULIS.X, PAULIS.Y, PAULIS.Z]
    widths = []
    for i in 1:3
        ε = 0.0
        err = pauls[i]
        F = 1.0
        while (ε < 0.5 && F >= thresh)
            noisy_drift = drift + ε * err
            noisy_sys = QuantumSystem(noisy_drift, drive)
            F = unitary_rollout_fidelity(traj, noisy_sys)
            ε += 0.0001
        end
        push!(widths, ε)
    end
    return widths
end


width_robustness (generic function with 1 method)

In [4]:
# Random.seed!(5)

# #Default
# def = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; Q_t=1.0)
# def_elapsed_time = @elapsed begin
# solve!(def, max_iter=250, print_level=5)
# end


# var_prob = UnitaryVariationalProblem(
#         varsys, U_goal, T, Δt;
#         robust_times=[[T], [T], [T]],
#         a_bound = 1.0,
#         dda_bound = 5000.0,
#         piccolo_options=piccolo_opts
# )

# var_elapsed_time = @elapsed begin
# solve!(var_prob, max_iter=500, print_level=5, options=IpoptOptions(eval_hessian=false))
# solve!(var_prob, max_iter=25, print_level=5)
# end



In [6]:
# method_trajs = [def.trajectory, var_prob.trajectory]
# prob_widths = [width_robustness(sys, traj) for traj in method_trajs]
# scrX = [space_curve_robustness(traj, GATES.H, PAULIS.X) for traj in method_trajs]
# scrY = [space_curve_robustness(traj, GATES.H, PAULIS.Y) for traj in method_trajs]
# scrZ = [space_curve_robustness(traj, GATES.H, PAULIS.Z) for traj in method_trajs]

# # Formatted output for robustness comparison
# method_names = ["Default", "Adjoint"]

# println("=" ^ 80)
# println("ROBUSTNESS COMPARISON")
# println("=" ^ 80)

# # Width Robustness Results
# println("\n📊 WIDTH ROBUSTNESS (Error Tolerance Thresholds)")
# println("-" ^ 60)
# println("Method          | X-Error  | Y-Error  | Z-Error  | Average")
# println("-" ^ 60)
# for (i, method) in enumerate(method_names)
#     widths = prob_widths[i]
#     avg_width = mean(widths)
#     @printf("%-15s | %8.4f | %8.4f | %8.4f | %8.4f\n", 
#             method, widths[1], widths[2], widths[3], avg_width)
# end

# # Space Curve Robustness Results
# println("\n🌌 SPACE CURVE ROBUSTNESS (Accumulated Error Sensitivity)")
# println("-" ^ 60)
# println("Method          | X-Sens   | Y-Sens   | Z-Sens   | Total")
# println("-" ^ 60)
# for (i, method) in enumerate(method_names)
#     x_sens = scrX[i]
#     y_sens = scrY[i]
#     z_sens = scrZ[i]
#     total_sens = sqrt(x_sens^2 + y_sens^2 + z_sens^2)
#     @printf("%-15s | %8.4f | %8.4f | %8.4f | %8.4f\n", 
#             method, x_sens, y_sens, z_sens, total_sens)
# end

# # Summary Rankings
# println("\n🏆 RANKINGS (1 = Best)")
# println("-" ^ 40)

# # Width robustness ranking (higher is better)
# width_averages = [mean(prob_widths[i]) for i in 1:5]
# width_ranks = sortperm(width_averages, rev=true)
# width_ranking = zeros(Int, 5)
# width_ranking[width_ranks] = 1:5

# # Space curve robustness ranking (lower is better)
# total_sensitivities = [sqrt(scrX[i]^2 + scrY[i]^2 + scrZ[i]^2) for i in 1:5]
# sens_ranks = sortperm(total_sensitivities)
# sens_ranking = zeros(Int, 5)
# sens_ranking[sens_ranks] = 1:5

# println("Method          | Width Rank | Sensitivity Rank")
# println("-" ^ 40)
# for (i, method) in enumerate(method_names)
#     @printf("%-15s |     %d      |       %d\n", 
#             method, width_ranking[i], sens_ranking[i])
# end

# println("\n💡 INTERPRETATION:")
# println("   Width Robustness: Higher values = more robust to errors")
# println("   Space Curve Sensitivity: Lower values = less sensitive to accumulated errors")
# println("=" ^ 80)

In [7]:
# using CairoMakie

# H_drive_add = H_drive

# f = Figure(size=(900, 800))
# ax1 = Axis(f[1, 1], title="X Error Robustness", xlabel="Error Strength (ε)", ylabel="Fidelity")
# ax2 = Axis(f[2, 1], title="Y Error Robustness", xlabel="Error Strength (ε)", ylabel="Fidelity")  
# ax3 = Axis(f[3, 1], title="Z Error Robustness", xlabel="Error Strength (ε)", ylabel="Fidelity")

# colors = Makie.wong_colors()
# method_names = ["Default", "Adjoint"]
# trajectories = [def.trajectory, var_prob.trajectory]

# εs = -0.2:0.01:0.2
# # X Error plots
# for (i, traj) in enumerate(trajectories)
#     ys = [unitary_rollout_fidelity(traj, QuantumSystem(sys.H.H_drift + ε * PAULIS.X, sys.H.H_drives)) for ε in εs]
#     lines!(ax1, εs, ys, label=method_names[i], color=colors[i], linewidth=2)
# end

# # Y Error plots  
# for (i, traj) in enumerate(trajectories)
#     ys = [unitary_rollout_fidelity(traj, QuantumSystem(sys.H.H_drift + ε * PAULIS.Y, sys.H.H_drives)) for ε in εs]
#     lines!(ax2, εs, ys, label=method_names[i], color=colors[i], linewidth=2)
# end

# # Z Error plots
# for (i, traj) in enumerate(trajectories)
#     ys = [unitary_rollout_fidelity(traj, QuantumSystem(sys.H.H_drift + ε * PAULIS.Z, sys.H.H_drives)) for ε in εs]
#     lines!(ax3, εs, ys, label=method_names[i], color=colors[i], linewidth=2)
# end

# # Add horizontal line at fidelity threshold
# # hlines!(ax1, [0.999], color=:red, linestyle=:dash, alpha=0.7)
# # hlines!(ax2, [0.999], color=:red, linestyle=:dash, alpha=0.7) 
# # hlines!(ax3, [0.999], color=:red, linestyle=:dash, alpha=0.7)

# # Set axis limits and formatting
# for ax in [ax1, ax2, ax3]
#     Makie.xlims!(ax, -0.2, 0.2)
#     Makie.ylims!(ax, 0.0, 1.01)
#     ax.ygridvisible = true
#     ax.xgridvisible = true
# end

# # Add a single legend for all subplots
# Legend(f[1:3, 2], ax1, "Methods", framevisible=true)

# # Add overall title
# Label(f[0, 1:2], "Quantum Control Robustness Comparison", 
# fontsize=20, font="bold")

# f

In [8]:
# dda_bounds = 10 .^ range(-1, 0, length=4)
round(T/2)

10.0

In [ ]:
# Define parameter ranges (logarithmic scale)
a_bounds = 10 .^ range(-1, 1, length=12)  # 1e0 to 1e1
dda_bounds = 10 .^ range(-1, 1, length=12)  # 1e-1 to 1e1
Random.seed!(5)
println("Parameter ranges:")
println("a_bounds: ", a_bounds)
println("dda_bounds: ", dda_bounds)

# Initialize results storage
trajectories = Matrix{Any}(undef, length(a_bounds), length(dda_bounds))
fidelities = zeros(length(a_bounds), length(dda_bounds))
convergence_times = zeros(length(a_bounds), length(dda_bounds))
final_costs = zeros(length(a_bounds), length(dda_bounds))

# Parameter sweep
println("\nStarting parameter sweep...")
for (i, a_bound) in enumerate(a_bounds)
    for (j, dda_bound) in enumerate(dda_bounds)
        println("Testing a_bound=$(Printf.@sprintf("%.3e", a_bound)), dda_bound=$(Printf.@sprintf("%.3e", dda_bound))")
        # Create variational problem with current parameters
        var_prob_test = UnitaryVariationalProblem(
            varsys, U_goal, T, Δt;
            robust_times=[[T], [T], [T]],
            a_bound = a_bound,
            dda_bound = dda_bound,
            piccolo_options=piccolo_opts
        )
        
        # Solve with timing
        elapsed_time = @elapsed begin
            solve!(var_prob_test, max_iter=750, print_level=0, 
                    options=IpoptOptions(eval_hessian=false))
            solve!(var_prob_test, max_iter=30, print_level=0)
        end
        
        # Calculate final fidelity
        fidelity = unitary_rollout_fidelity(var_prob_test.trajectory, sys)
        # Store results
        trajectories[i, j] = var_prob_test.trajectory
        fidelities[i, j] = fidelity
        convergence_times[i, j] = elapsed_time
        Z_vec = vec(var_prob_test.trajectory)
        final_costs[i, j] = var_prob_test.objective.L(Z_vec)
        
        println("  -> Fidelity: $(Printf.@sprintf("%.6f", fidelity)), Time: $(Printf.@sprintf("%.2f", elapsed_time))s")
    end
end

Parameter ranges:
a_bounds: 

In [ ]:
# Create comprehensive visualization
f = Figure(size=(1400, 1000))

# 1. 3D Surface Plot
ax3d = Axis3(f[1, 1:2], 
    title="Fidelity vs a_bound and dda_bound",
    xlabel="log₁₀(a_bound)", 
    ylabel="log₁₀(dda_bound)", 
    zlabel="Fidelity",
    aspect=(1, 1, 0.5)
)

# Create meshgrid for 3D plot
x_3d = repeat(log10.(a_bounds), 1, length(dda_bounds))
y_3d = repeat(log10.(dda_bounds)', length(a_bounds), 1)
z_3d = fidelities

# Create 3D surface
Makie.surface!(ax3d, x_3d, y_3d, z_3d, colormap=:viridis, alpha=0.8)

# Add contour lines on the surface
Makie.contour3d!(ax3d, x_3d, y_3d, z_3d, levels=10, linewidth=1, alpha=0.6)

# 2. 2D Heatmap of Fidelity
ax_heat = Axis(f[2, 1], 
    title="Fidelity Heatmap",
    xlabel="log₁₀(a_bound)", 
    ylabel="log₁₀(dda_bound)"
)

# Fix: Use correct dimensions for heatmap and contour
hm = Makie.heatmap!(ax_heat, log10.(a_bounds), log10.(dda_bounds), fidelities, 
             colormap=:viridis, colorrange=(0.95, 1.0))
Colorbar(f[2, 2], hm, label="Fidelity")

# Add contour lines - fix dimension matching
Makie.contour!(ax_heat, log10.(a_bounds), log10.(dda_bounds), fidelities, 
        levels=[0.99, 0.995, 0.999], color=:white, linewidth=1.5)

# 3. Cross-sections at different dda_bound values
ax_cross1 = Axis(f[1, 3], 
    title="Fidelity vs a_bound (different dda_bound)",
    xlabel="log₁₀(a_bound)", 
    ylabel="Fidelity"
)

# Select a few representative dda_bound values for cross-sections
n_dda = length(dda_bounds)
dda_indices = [max(1, round(Int, n_dda/4)), 
               max(1, round(Int, n_dda/2)), 
               max(1, round(Int, 3*n_dda/4)), 
               n_dda]  # Ensure valid indices
colors_cross = [:red, :blue, :green, :orange]

for (idx, dda_idx) in enumerate(dda_indices)
    if dda_idx <= n_dda  # Safety check
        lines!(ax_cross1, log10.(a_bounds), fidelities[:, dda_idx], 
               label="dda_bound = $(Printf.@sprintf("%.2e", dda_bounds[dda_idx]))",
               color=colors_cross[idx], linewidth=2)
    end
end

axislegend(ax_cross1, position=:rb)

# 4. Cross-sections at different a_bound values
ax_cross2 = Axis(f[2, 3], 
    title="Fidelity vs dda_bound (different a_bound)",
    xlabel="log₁₀(dda_bound)", 
    ylabel="Fidelity"
)

# Select a few representative a_bound values for cross-sections
n_a = length(a_bounds)
a_indices = [max(1, round(Int, n_a/4)), 
             max(1, round(Int, n_a/2)), 
             max(1, round(Int, 3*n_a/4)), 
             n_a]  # Ensure valid indices

for (idx, a_idx) in enumerate(a_indices)
    if a_idx <= n_a  # Safety check
        lines!(ax_cross2, log10.(dda_bounds), fidelities[a_idx, :], 
               label="a_bound = $(Printf.@sprintf("%.2e", a_bounds[a_idx]))",
               color=colors_cross[idx], linewidth=2)
    end
end

axislegend(ax_cross2, position=:rb)

# Add overall title
Label(f[0, 1:3], "Variational Quantum Control: Parameter Sensitivity Analysis", 
      fontsize=20, font="bold")

# Display the figure
f


In [ ]:

# Print summary statistics
println("\n" * "="^60)
println("SUMMARY STATISTICS")
println("="^60)

max_fidelity = maximum(fidelities)
max_idx = argmax(fidelities)
best_a = a_bounds[max_idx[1]]
best_dda = dda_bounds[max_idx[2]]

println("Best fidelity: $(Printf.@sprintf("%.6f", max_fidelity))")
println("Best parameters: a_bound = $(Printf.@sprintf("%.3e", best_a)), dda_bound = $(Printf.@sprintf("%.3e", best_dda))")

# Find parameters giving fidelity > 0.999
high_fidelity_mask = fidelities .> 0.999
n_high_fidelity = sum(high_fidelity_mask)
println("Number of parameter combinations with fidelity > 0.999: $n_high_fidelity / $(length(fidelities))")

if n_high_fidelity > 0
    println("High fidelity parameter ranges:")
    high_a_indices = findall(any(high_fidelity_mask, dims=2)[:, 1])
    high_dda_indices = findall(any(high_fidelity_mask, dims=1)[1, :])
    
    if !isempty(high_a_indices)
        println("  a_bound: $(Printf.@sprintf("%.3e", minimum(a_bounds[high_a_indices]))) to $(Printf.@sprintf("%.3e", maximum(a_bounds[high_a_indices])))")
    end
    if !isempty(high_dda_indices)
        println("  dda_bound: $(Printf.@sprintf("%.3e", minimum(dda_bounds[high_dda_indices]))) to $(Printf.@sprintf("%.3e", maximum(dda_bounds[high_dda_indices])))")
    end
end

# Analyze trade-offs
println("\nTRADE-OFF ANALYSIS:")
println("-"^40)

# Effect of a_bound (averaging over dda_bound)
avg_fidelity_vs_a = mean(fidelities, dims=2)[:, 1]
println("Average fidelity vs a_bound:")
for (i, a_bound) in enumerate(a_bounds)
    println("  a_bound = $(Printf.@sprintf("%.3e", a_bound)): avg fidelity = $(Printf.@sprintf("%.4f", avg_fidelity_vs_a[i]))")
end

# Effect of dda_bound (averaging over a_bound)
avg_fidelity_vs_dda = mean(fidelities, dims=1)[1, :]
println("\nAverage fidelity vs dda_bound:")
for (j, dda_bound) in enumerate(dda_bounds)
    println("  dda_bound = $(Printf.@sprintf("%.3e", dda_bound)): avg fidelity = $(Printf.@sprintf("%.4f", avg_fidelity_vs_dda[j]))")
end

In [ ]:
# Create comprehensive visualization
f = Figure(size=(1400, 1000))

# 1. 3D Surface Plot
ax3d = Axis3(f[1, 1:2], 
    title="Fidelity vs a_bound and dda_bound",
    xlabel="log₁₀(a_bound)", 
    ylabel="log₁₀(dda_bound)", 
    zlabel="Fidelity",
    aspect=(1, 1, 0.5)
)

# Create meshgrid for 3D plot
x_3d = repeat(log10.(a_bounds), 1, length(dda_bounds))
y_3d = repeat(log10.(dda_bounds)', length(a_bounds), 1)
z_3d = fidelities

# Create 3D surface
Makie.surface!(ax3d, x_3d, y_3d, z_3d, colormap=:viridis, alpha=0.8)

# Add contour lines on the surface
Makie.contour3d!(ax3d, x_3d, y_3d, z_3d, levels=10, linewidth=1, alpha=0.6)

# 2. 2D Heatmap of Fidelity
ax_heat = Axis(f[2, 1], 
    title="Fidelity Heatmap",
    xlabel="log₁₀(a_bound)", 
    ylabel="log₁₀(dda_bound)"
)

# Fix: Use correct dimensions for heatmap and contour
hm = Makie.heatmap!(ax_heat, log10.(a_bounds), log10.(dda_bounds), fidelities, 
             colormap=:viridis, colorrange=(0.95, 1.0))
Colorbar(f[2, 2], hm, label="Fidelity")

# Add contour lines - fix dimension matching
Makie.contour!(ax_heat, log10.(a_bounds), log10.(dda_bounds), fidelities, 
        levels=[0.99, 0.995, 0.999], color=:white, linewidth=1.5)

# 3. Cross-sections at different dda_bound values
ax_cross1 = Axis(f[1, 3], 
    title="Fidelity vs a_bound (different dda_bound)",
    xlabel="log₁₀(a_bound)", 
    ylabel="Fidelity"
)

# Select a few representative dda_bound values for cross-sections
n_dda = length(dda_bounds)
dda_indices = [max(1, round(Int, n_dda/4)), 
               max(1, round(Int, n_dda/2)), 
               max(1, round(Int, 3*n_dda/4)), 
               n_dda]  # Ensure valid indices
colors_cross = [:red, :blue, :green, :orange]

for (idx, dda_idx) in enumerate(dda_indices)
    if dda_idx <= n_dda  # Safety check
        lines!(ax_cross1, log10.(a_bounds), fidelities[:, dda_idx], 
               label="dda_bound = $(Printf.@sprintf("%.2e", dda_bounds[dda_idx]))",
               color=colors_cross[idx], linewidth=2)
    end
end

axislegend(ax_cross1, position=:rb)

# 4. Cross-sections at different a_bound values
ax_cross2 = Axis(f[2, 3], 
    title="Fidelity vs dda_bound (different a_bound)",
    xlabel="log₁₀(dda_bound)", 
    ylabel="Fidelity"
)

# Select a few representative a_bound values for cross-sections
n_a = length(a_bounds)
a_indices = [max(1, round(Int, n_a/4)), 
             max(1, round(Int, n_a/2)), 
             max(1, round(Int, 3*n_a/4)), 
             n_a]  # Ensure valid indices

for (idx, a_idx) in enumerate(a_indices)
    if a_idx <= n_a  # Safety check
        lines!(ax_cross2, log10.(dda_bounds), fidelities[a_idx, :], 
               label="a_bound = $(Printf.@sprintf("%.2e", a_bounds[a_idx]))",
               color=colors_cross[idx], linewidth=2)
    end
end

axislegend(ax_cross2, position=:rb)

# Add overall title
Label(f[0, 1:3], "Variational Quantum Control: Parameter Sensitivity Analysis", 
      fontsize=20, font="bold")

# Display the figure
f
